<a href="https://colab.research.google.com/github/manushi0304/Diabetic_Retinopathy/blob/main/ComparisonEfficeinet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# REPLACE arithmetic for ALL heads:
# (Base FP16 TFLite - built-in FP16 Dense head) + head (+ preproc)
# Model: EfficientNetV2B0 (penultimate d=1280), C=5
# Heads covered: Linear SVM, RBF SVM, Random Forest, kNN
# ==========================================

# 0) Mount (safe to re-run)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass

import os, csv
from pathlib import Path

# ===== 1) CONFIG =====
# Path to your EfficientNetV2B0 FP16 TFLite (or fallback MB will be used)
TFLITE_PATH = "/content/drive/MyDrive/DiabeticProject/tflite/EfficientNetV2B0_model_fp16.tflite"  # <- change if needed
BASE_MB_FALLBACK = 12.3     # used only if TFLITE_PATH doesn't exist (approx FP16 size)

# Built-in head details to remove (Dense(d×C)+bias in FP16)
d = 1280    # feature dimension from EfficientNetV2B0 penultimate layer
C = 5       # number of classes
HEAD_DTYPE = "float16"  # built-in head dtype inside FP16 TFLite

# External head params
# SVM / RF / kNN knobs
N = 8000               # kNN: stored feature vectors
m = 1200               # RBF-SVM: total support vectors across OvR/OvO
T = 100                # RandomForest: number of trees
nodes_per_tree = 511   # avg nodes per tree

# Preprocessing (count once per scenario)
INCLUDE_SCALER = True
INCLUDE_PCA    = False
d_pca          = 256   # only used if INCLUDE_PCA=True

# Dtypes for external heads
EXT_EMB_DTYPE  = "float16"  # embeddings/support vectors storage (kNN/RBF-SVM)
EXT_WTS_DTYPE  = "float32"  # linear SVM weights & biases
LABEL_BYTES    = 1

# Output CSV path
OUT_CSV = "/content/drive/MyDrive/DiabeticProject/saved_models/hybrid_size_replace_allheads_effv2b0.csv"

# ===== 2) Helpers =====
def bytes_per_float(dtype:str)->int:
    x = dtype.lower()
    if x in ("float16","fp16","half"): return 2
    if x in ("float32","fp32","single"): return 4
    if x in ("float64","fp64","double"): return 8
    raise ValueError(f"Unsupported dtype: {dtype}")

def size_mb(nbytes:int)->float:
    return nbytes / (1024.0 * 1024.0)

def get_base_mb(path:str|None, fallback_mb:float)->float:
    if path and os.path.exists(path):
        return os.path.getsize(path) / (1024.0*1024.0)
    return fallback_mb

# Built-in Dense(d×C)+bias to REMOVE (assume head stored in FP16)
def builtin_dense_head_bytes(d:int, C:int, head_dtype:str)->int:
    bpf = bytes_per_float(head_dtype)
    return (d*C + C) * bpf

# External heads to ADD
def scaler_bytes(d:int)->int:
    # mean + std (float32)
    return 2 * d * bytes_per_float("float32")

def pca_bytes(d_orig:int, d_pca:int)->int:
    # components(d_pca x d_orig) + mean(d_orig) as float32
    bf = bytes_per_float("float32")
    return d_pca*d_orig*bf + d_orig*bf

def linear_svm_ovr_bytes(C:int, d:int, dtype:str="float32")->int:
    bf = bytes_per_float(dtype)
    return (C*d + C) * bf  # W + b

def rbf_svm_bytes(m:int, d:int, C:int, sv_dtype:str="float32")->int:
    bf = bytes_per_float(sv_dtype)
    # support vectors + dual coeffs (OvR approx m*C) + biases(C)
    return m*d*bf + m*C*bf + C*bf

def knn_bytes(N:int, d:int, emb_dtype:str="float16", label_bytes:int=1)->int:
    bf = bytes_per_float(emb_dtype)
    return N*d*bf + N*label_bytes

def random_forest_bytes(T:int, nodes_per_tree:int, bytes_per_node:int=32)->int:
    return T * nodes_per_tree * bytes_per_node

# ===== 3) Base and "backbone-only" via subtraction =====
base_mb = get_base_mb(TFLITE_PATH, BASE_MB_FALLBACK)
head_rm_mb = size_mb(builtin_dense_head_bytes(d, C, HEAD_DTYPE))
backbone_only_mb = max(0.0, base_mb - head_rm_mb)

print("===== BASELINE (EfficientNetV2B0) =====")
print(f"Base FP16 TFLite       : {base_mb:.3f} MB  ({Path(TFLITE_PATH).name if os.path.exists(TFLITE_PATH) else 'fallback'})")
print(f"Remove FP16 Dense(d×C) : -{head_rm_mb:.3f} MB [d={d}, C={C}, dtype={HEAD_DTYPE}]")
print(f"Backbone-only (est.)   :  {backbone_only_mb:.3f} MB")

# Preprocessing shared term
prep_bytes = 0
if INCLUDE_SCALER:
    prep_bytes += scaler_bytes(d)
if INCLUDE_PCA:
    prep_bytes += pca_bytes(d, d_pca)
prep_mb = size_mb(prep_bytes)

# ===== 4) Scenarios: each head separately (REPLACE arithmetic) =====
scenarios = []

def add_scenario(name, add_bytes):
    total_mb = backbone_only_mb + size_mb(add_bytes + (prep_bytes if add_bytes>0 else 0))
    scenarios.append({
        "scenario": name,
        "base_fp16_mb": round(base_mb, 6),
        "removed_head_mb": round(head_rm_mb, 6),
        "backbone_only_mb": round(backbone_only_mb, 6),
        "head_mb": round(size_mb(add_bytes), 6),
        "preproc_mb": round(prep_mb if add_bytes>0 else 0.0, 6),
        "total_mb": round(total_mb, 6),
        "d": d, "C": C, "N": N, "m": m, "T": T, "nodes_per_tree": nodes_per_tree,
        "head_dtype_removed": HEAD_DTYPE,
        "ext_emb_dtype": EXT_EMB_DTYPE,
        "ext_wts_dtype": EXT_WTS_DTYPE,
        "include_scaler": INCLUDE_SCALER,
        "include_pca": INCLUDE_PCA,
        "tflite_path": TFLITE_PATH if os.path.exists(TFLITE_PATH) else "(fallback)",
    })

# Linear SVM
lin_bytes = linear_svm_ovr_bytes(C, d, dtype=EXT_WTS_DTYPE)
add_scenario("REPLACE → Linear SVM", lin_bytes)

# RBF SVM
rbf_bytes = rbf_svm_bytes(m, d, C, sv_dtype=EXT_EMB_DTYPE)
add_scenario("REPLACE → RBF SVM", rbf_bytes)

# Random Forest
rf_bytes = random_forest_bytes(T, nodes_per_tree, 32)
add_scenario("REPLACE → Random Forest", rf_bytes)

# kNN
knn_b = knn_bytes(N, d, emb_dtype=EXT_EMB_DTYPE, label_bytes=LABEL_BYTES)
add_scenario("REPLACE → kNN", knn_b)

# ===== 5) Print nicely =====
print("\n===== HYBRID SIZE via REPLACE (each head) — EfficientNetV2B0 =====")
for s in scenarios:
    print(f"\n{s['scenario']}")
    print(f"Backbone-only (est.)   : {s['backbone_only_mb']:.3f} MB")
    print(f"(+) Head               : {s['head_mb']:.3f} MB")
    if s['head_mb'] > 0:
        print(f"(+) Preprocessing      : {s['preproc_mb']:.3f} MB")
    print(f"{'TOTAL':25} : {s['total_mb']:.3f} MB")

# ===== 6) Save CSV =====
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
with open(OUT_CSV, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=list(scenarios[0].keys()))
    writer.writeheader()
    writer.writerows(scenarios)
print(f"\n[saved] {OUT_CSV}")

print("\nNotes:")
print("- These totals use REPLACE arithmetic: (base - FP16 Dense head) + head (+ shared preproc).")
print("- The removed head is ~0.013 MB for d=1280, C=5 (tiny), so totals stay close to base FP16.")
print("- To get totals significantly < base FP16, use an INT8 backbone or a smaller/pruned network.")


Mounted at /content/drive
===== BASELINE (EfficientNetV2B0) =====
Base FP16 TFLite       : 12.943 MB  (EfficientNetV2B0_model_fp16.tflite)
Remove FP16 Dense(d×C) : -0.012 MB [d=1280, C=5, dtype=float16]
Backbone-only (est.)   :  12.931 MB

===== HYBRID SIZE via REPLACE (each head) — EfficientNetV2B0 =====

REPLACE → Linear SVM
Backbone-only (est.)   : 12.931 MB
(+) Head               : 0.024 MB
(+) Preprocessing      : 0.010 MB
TOTAL                     : 12.965 MB

REPLACE → RBF SVM
Backbone-only (est.)   : 12.931 MB
(+) Head               : 2.941 MB
(+) Preprocessing      : 0.010 MB
TOTAL                     : 15.881 MB

REPLACE → Random Forest
Backbone-only (est.)   : 12.931 MB
(+) Head               : 1.559 MB
(+) Preprocessing      : 0.010 MB
TOTAL                     : 14.500 MB

REPLACE → kNN
Backbone-only (est.)   : 12.931 MB
(+) Head               : 19.539 MB
(+) Preprocessing      : 0.010 MB
TOTAL                     : 32.479 MB

[saved] /content/drive/MyDrive/DiabeticProject